<a href="https://colab.research.google.com/github/Impana33/Tree-canopy-Detection/blob/main/GeoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==== (optional) check GPU ====
!nvidia-smi

# ==== core build tools ====
!pip -q install -U pip setuptools wheel

# ==== PyTorch (GPU). If no GPU, see CPU option below ====
!pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# ==== system deps for rtree (spatial index) ====
!apt-get -qq update
!apt-get -qq install -y libspatialindex-dev

# ==== geo/python stack ====
!pip -q install rasterio shapely pyproj fiona rtree geopandas
!pip -q install scikit-image scipy numpy pandas tqdm

# ==== DeepForest (GeoAI for canopy detection) ====
!pip -q install deepforest

# ---- (CPU-only PyTorch alternative; use this instead of the GPU line above) ----
# !pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cpu


/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 126666 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ====== EDIT THIS ONLY IF YOUR FOLDER NAME IS DIFFERENT ======
BASE = "/content/drive/MyDrive/GeoAI"   # <-- use the exact case of your Drive folder
# If your folder is actually "Geoai", then do:
# BASE = "/content/drive/MyDrive/Geoai"
# =============================================================

# --- INPUTS ---
RGB_PATH = f"{BASE}/Test1_Trees/Test1_Trees.tif"                   # RGB (bands 1–3)
DSM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DSM.tif"
DTM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DTM.tif"

# --- CHM build outputs ---
CHM_BUILD_DIR   = f"{BASE}/Test1_DSM_DTM"
CHM_RAW_PATH    = f"{CHM_BUILD_DIR}/CHM_raw.tif"
CHM_SMOOTH_PATH = f"{CHM_BUILD_DIR}/CHM_smooth.tif"
CHM_FIXED_PATH  = f"{CHM_BUILD_DIR}/CHM_smooth_fixed.tif"  # final CHM

# --- Extraction outputs ---
OUT_DIR    = f"{BASE}/out_colab"
CROWNS_GPKG = f"{OUT_DIR}/Tree_Crowns.gpkg"
POINTS_GPKG = f"{OUT_DIR}/Tree_Points.gpkg"
POINTS_CSV  = f"{OUT_DIR}/Tree_Points.csv"

# --- Params ---
DF_SCORE       = 0.20   # DeepForest score threshold
MIN_HEIGHT_M   = 2.0    # CHM min height (m)
GAUSS_SIGMA_PX = 1.0    # extra CHM smoothing
MIN_CROWN_M2   = 0.30   # min crown area (m²)

import os
os.makedirs(OUT_DIR, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import rasterio as rio
from rasterio.warp import reproject, Resampling
from scipy.ndimage import gaussian_filter

with rio.open(DSM_PATH) as dsm_src:
    dsm = dsm_src.read(1).astype("float32")
    prof = dsm_src.profile
    tfm  = dsm_src.transform
    crs  = dsm_src.crs

with rio.open(DTM_PATH) as dtm_src:
    dtm = dtm_src.read(1).astype("float32")
    dtm_prof = dtm_src.profile

# handle NoData
dsm[dsm <= -9999] = np.nan
dtm[dtm <= -9999] = np.nan

# resample DTM to DSM grid if needed
same_grid = (prof["crs"]==dtm_prof["crs"] and prof["transform"]==dtm_prof["transform"] and dsm.shape==dtm.shape)
if not same_grid:
    dtm_res = np.empty_like(dsm, dtype="float32")
    reproject(
        source=dtm, destination=dtm_res,
        src_transform=dtm_prof["transform"], src_crs=dtm_prof["crs"],
        dst_transform=prof["transform"],   dst_crs=prof["crs"],
        resampling=Resampling.bilinear
    )
else:
    dtm_res = dtm

# CHM compute + clean
chm = dsm - dtm_res
chm = np.where(np.isfinite(chm), chm, 0.0).astype("float32")
chm[chm < 0] = 0.0

# save raw CHM
out_prof = prof.copy()
out_prof.update(dtype="float32", count=1, compress="LZW", nodata=0)
with rio.open(CHM_RAW_PATH, "w", **out_prof) as dst:
    dst.write(chm, 1)

# smooth + save fixed CHM
chm_s = gaussian_filter(chm, sigma=1.0)  # light smoothing
chm_s[chm_s < 0] = 0.0
with rio.open(CHM_FIXED_PATH, "w", **out_prof) as dst:
    dst.write(chm_s, 1)

print("CHM written:", CHM_FIXED_PATH)
print("CHM stats min/max:", float(chm_s.min()), float(chm_s.max()))


CHM written: /content/drive/MyDrive/GeoAI/Test1_DSM_DTM/CHM_smooth_fixed.tif
CHM stats min/max: 0.0 8.882564544677734


In [ ]:
# Uninstall any v2 albumentations
!pip -q uninstall -y albumentations

# Install a compatible v1 release (works with DeepForest)
!pip -q install "albumentations<2.0"  # e.g., 1.4.3

# (Optional but safe) make sure deepforest is present
!pip -q install deepforest

# Restart the runtime so the old import cache is gone
import os, sys
os.kill(os.getpid(), 9)


In [ ]:
CHM_PATH =  r"/content/drive/MyDrive/GeoAI/Test1_DSM_DTM/CHM_smooth_fixed.tif"# reuse the variable from above
import os
print("Using CHM:", CHM_PATH)


Using CHM: /content/drive/MyDrive/GeoAI/Test1_DSM_DTM/CHM_smooth_fixed.tif


In [ ]:
print(os.path.exists(RGB_PATH), RGB_PATH)
print(os.path.exists(DSM_PATH), DSM_PATH)
print(os.path.exists(DTM_PATH), DTM_PATH)
print(os.path.exists(CHM_BUILD_DIR), CHM_BUILD_DIR)


False /content/drive/MyDrive/GeoAI/Test1_Trees/Test1_Trees.tif
True /content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees_DSM.tif
True /content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees_DTM.tif
True /content/drive/MyDrive/GeoAI/Test1_DSM_DTM


In [ ]:
BASE = "/content/drive/MyDrive/Geoai"   # or /MyDrive/GeoAI  (case-sensitive!)

RGB_PATH = f"{BASE}/Test1_Trees/Test1_Trees.tif"
DSM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DSM.tif"
DTM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DTM.tif"

OUT_DIR  = f"{BASE}/out_colab"


In [ ]:
# (paste the CHM builder you ran earlier)
# It writes: CHM_smooth_fixed.tif
from pathlib import Path
CHM_BUILD_DIR = f"{BASE}/Test1_DSM_DTM"
CHM_FIXED_PATH = f"{CHM_BUILD_DIR}/CHM_smooth_fixed.tif"


In [ ]:
import os, numpy as np, rasterio as rio
from deepforest import main

# --- use the exact path you provided ---
RGB_PATH = "/content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees.tif"

# 1) sanity check
print("Exists? ", os.path.exists(RGB_PATH), " -> ", RGB_PATH)
if not os.path.exists(RGB_PATH):
    raise FileNotFoundError(f"Path not found: {RGB_PATH}\nTip: run `!ls -lah '/content/drive/MyDrive/GeoAI/Test1_DSM_DTM'` to inspect.")

# 2) read bands (expects RGB in bands 1-3)
with rio.open(RGB_PATH) as src:
    print("Band count:", src.count, "dtype:", src.dtypes, "CRS:", src.crs)
    if src.count < 3:
        raise ValueError("This file has <3 bands. DeepForest needs 3-channel RGB.")
    R = src.read(1).astype("float32"); R[R<0]=0
    G = src.read(2).astype("float32"); G[G<0]=0
    B = src.read(3).astype("float32"); B[B<0]=0
    transform = src.transform
    crs = src.crs

# If your image is 0–1 range, scale up; otherwise leave as is
if R.max() <= 1.0 and G.max() <= 1.0 and B.max() <= 1.0:
    R *= 255; G *= 255; B *= 255

rgb_img = np.dstack([R,G,B]).astype("float32")
print("RGB shape:", rgb_img.shape, "max:", float(rgb_img.max()))

# 3) run DeepForest
DF_SCORE = 0.20  # tweak later if needed
model = main.deepforest()
model.use_release()

df_boxes = model.predict_image(rgb_img, return_plot=False)
df_boxes = df_boxes[df_boxes["score"] >= DF_SCORE].reset_index(drop=True)
print(f"Detections kept @score≥{DF_SCORE}: {len(df_boxes)}")
df_boxes.head()



/usr/local/lib/python3.12/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Exists?  True  ->  /content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees.tif
Band count: 4 dtype: ('uint8', 'uint8', 'uint8', 'uint8') CRS: EPSG:32644
RGB shape: (1758, 2300, 3) max: 255.0
Reading config file: /usr/local/lib/python3.12/dist-packages/deepforest/data/deepforest_config.yml
Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth


100%|██████████| 130M/130M [00:01<00:00, 121MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Reading config file: /usr/local/lib/python3.12/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

Detections kept @score≥0.2: 2


,xmin,ymin,xmax,ymax,label,score,geometry
0,555.0,268.0,795.0,476.0,Tree,0.28920,"POLYGON ((795 268, 795 476, 555 476, 555 268, ..."
1,1859.0,120.0,2015.0,278.0,Tree,0.26772,"POLYGON ((2015 120, 2015 278, 1859 278, 1859 1..."


In [ ]:
# ====== EDIT THESE ======
BASE = "/content/drive/MyDrive/GeoAI"  # or /Geoai (match the exact case)

RGB_PATH = f"{BASE}/Test1_Trees/Test1_Trees.tif"              # RGB or RGB+NIR (bands 1-3, optional 4=NIR)
# If you already built a CHM:
CHM_PATH = f"{BASE}/Test1_DSM_DTM/CHM_smooth_fixed.tif"       # set to None if you don't have a CHM

# If you DON'T have a CHM but DO have DSM/DTM, set these (else leave as None)
DSM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DSM.tif"        # or None
DTM_PATH = f"{BASE}/Test1_DSM_DTM/Test1_Trees_DTM.tif"        # or None

OUT_DIR  = f"{BASE}/out_from_thresholds"
POINTS_GPKG = f"{OUT_DIR}/Tree_Points.gpkg"
CROWNS_GPKG = f"{OUT_DIR}/Tree_Crowns.gpkg"
POINTS_CSV  = f"{OUT_DIR}/Tree_Points.csv"

# ---- THRESHOLDS / TUNING ----
NDVI_MIN      = 0.25   # vegetation mask; set None to skip NDVI test
MIN_HEIGHT_M  = 2.0    # CHM minimum height to keep
GAUSS_SIGMA_PX= 1.0    # extra CHM smoothing before local maxima (0–1.5)
CROWN_DIAM_M  = 2.0    # expected crown diameter -> local maxima window
MIN_CROWN_M2  = 0.30   # drop tiny crowns after watershed
# =============================


In [ ]:
import os
from pathlib import Path
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.warp import reproject, Resampling
from rasterio.transform import xy
from rasterio import features
from shapely.geometry import Point, shape
from scipy.ndimage import gaussian_filter, maximum_filter
from skimage.segmentation import watershed

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def build_chm_from_dsm_dtm(DSM_PATH, DTM_PATH):
    with rio.open(DSM_PATH) as dsm_src:
        dsm = dsm_src.read(1).astype("float32"); dsm[dsm<=-9999]=np.nan
        prof = dsm_src.profile; tf = dsm_src.transform; crs = dsm_src.crs
    with rio.open(DTM_PATH) as dtm_src:
        dtm = dtm_src.read(1).astype("float32"); dtm[dtm<=-9999]=np.nan
        dtm_prof = dtm_src.profile
    same = (prof["crs"]==dtm_prof["crs"] and prof["transform"]==dtm_prof["transform"] and dsm.shape==dtm.shape)
    if not same:
        dtm_res = np.empty_like(dsm, dtype="float32")
        reproject(dtm, dtm_res, src_transform=dtm_prof["transform"], src_crs=dtm_prof["crs"],
                  dst_transform=prof["transform"], dst_crs=prof["crs"], resampling=Resampling.bilinear)
    else:
        dtm_res = dtm
    chm = dsm - dtm_res
    chm = np.where(np.isfinite(chm), chm, 0).astype("float32")
    chm[chm<0]=0
    return chm, prof, tf, crs


In [ ]:
import os
from pathlib import Path
import numpy as np
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.warp import reproject, Resampling
from rasterio.transform import xy
from rasterio import features
from shapely.geometry import Point, shape
from scipy.ndimage import gaussian_filter, maximum_filter
from skimage.segmentation import watershed

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def build_chm_from_dsm_dtm(DSM_PATH, DTM_PATH):
    with rio.open(DSM_PATH) as dsm_src:
        dsm = dsm_src.read(1).astype("float32"); dsm[dsm<=-9999]=np.nan
        prof = dsm_src.profile; tf = dsm_src.transform; crs = dsm_src.crs
    with rio.open(DTM_PATH) as dtm_src:
        dtm = dtm_src.read(1).astype("float32"); dtm[dtm<=-9999]=np.nan
        dtm_prof = dtm_src.profile
    same = (prof["crs"]==dtm_prof["crs"] and prof["transform"]==dtm_prof["transform"] and dsm.shape==dtm.shape)
    if not same:
        dtm_res = np.empty_like(dsm, dtype="float32")
        reproject(dtm, dtm_res, src_transform=dtm_prof["transform"], src_crs=dtm_prof["crs"],
                  dst_transform=prof["transform"], dst_crs=prof["crs"], resampling=Resampling.bilinear)
    else:
        dtm_res = dtm
    chm = dsm - dtm_res
    chm = np.where(np.isfinite(chm), chm, 0).astype("float32")
    chm[chm<0]=0
    return chm, prof, tf, crs


In [ ]:
RGB_PATH = r"/content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees.tif"
with rio.open(RGB_PATH) as src:
    count = src.count
    R = src.read(1).astype("float32"); R[R<0]=0
    G = src.read(2).astype("float32"); G[G<0]=0
    B = src.read(3).astype("float32"); B[B<0]=0
    transform = src.transform; crs = src.crs

NIR = None
if count >= 4:
    with rio.open(RGB_PATH) as src:
        NIR = src.read(4).astype("float32"); NIR[NIR<0]=0

# NDVI mask
if NIR is not None and NDVI_MIN is not None:
    denom = (NIR + R); denom[denom==0]=1e-6
    NDVI = (NIR - R) / denom
    veg_mask = NDVI >= NDVI_MIN
else:
    veg_mask = None

print("Bands:", count, "| NDVI used:", veg_mask is not None)


Bands: 4 | NDVI used: True


In [ ]:
if CHM_PATH and os.path.exists(CHM_PATH):
    with rio.open(CHM_PATH) as chm_src:
        chm = chm_src.read(1).astype("float32"); chm[chm<0]=0
        chm_tf = chm_src.transform; chm_crs = chm_src.crs
        px_w, px_h = abs(chm_tf.a), abs(chm_tf.e)
else:
    assert DSM_PATH and DTM_PATH, "Provide CHM_PATH or DSM/DTM to build CHM."
    chm, chm_prof, chm_tf, chm_crs = build_chm_from_dsm_dtm(DSM_PATH, DTM_PATH)
    px_w, px_h = abs(chm_tf.a), abs(chm_tf.e)

# align masks: assume same grid as RGB; if not identical, it’s fine—the CHM is primary for points
if GAUSS_SIGMA_PX and GAUSS_SIGMA_PX > 0:
    chm_s = gaussian_filter(chm, sigma=GAUSS_SIGMA_PX)
else:
    chm_s = chm

# base canopy mask by height
height_mask = chm_s >= MIN_HEIGHT_M
# combine with NDVI mask if available
if veg_mask is not None:
    # If NDVI grid ≠ CHM grid, you can ignore NDVI or resample NDVI; here we just AND when shapes match
    if veg_mask.shape == chm_s.shape:
        mask = height_mask & veg_mask
    else:
        mask = height_mask
else:
    mask = height_mask

print("CHM stats:", float(chm_s.min()), float(chm_s.max()))


CHM stats: 0.0 8.826957702636719


In [ ]:
# window size ≈ expected crown diameter / pixel size
PIX_M = px_w  # pixel size in meters (assumes square pixels)
win_px = max(3, int(round(CROWN_DIAM_M / PIX_M)))
if win_px % 2 == 0: win_px += 1  # make it odd

# local maxima within the mask
filt = maximum_filter(chm_s, size=win_px, mode="nearest")
peaks = (chm_s == filt) & mask

ry, rx = np.where(peaks)
print("Detected peak count:", len(rx))


Detected peak count: 165


In [ ]:
# markers from peaks
markers = np.zeros_like(chm_s, dtype=np.int32)
for i,(r,c) in enumerate(zip(ry,rx), start=1):
    markers[r,c] = i

labels = watershed(-chm_s, markers=markers, mask=mask)

# remove tiny/huge crowns by area
px_area = px_w * px_h
min_px = max(1, int(round(MIN_CROWN_M2 / px_area)))
counts = np.bincount(labels.ravel())
small_ids = np.where(counts < min_px)[0]
labels[np.isin(labels, small_ids)] = 0

if isinstance(MAXB_CROWN_M2:=globals().get("MAX_CROWN_M2"), float) and MAXB_CROWN_M2 is not None:
    max_px = int(round(MAXB_CROWN_M2 / px_area))
    big_ids = np.where(counts > max_px)[0]
    labels[np.isin(labels, big_ids)] = 0


In [ ]:
# points (centroids of peaks in map coords)
xs, ys = [], []
for r,c in zip(ry,rx):
    x,y = xy(chm_tf, r, c)
    xs.append(x); ys.append(y)

gdf_pts = gpd.GeoDataFrame({"tree_id": np.arange(1, len(xs)+1)},
                           geometry=gpd.points_from_xy(xs, ys), crs=chm_crs)

# vectorize crowns
geoms, ids = [], []
for geom, val in features.shapes(labels.astype("int32"), mask=(labels>0), transform=chm_tf):
    if val == 0: continue
    geoms.append(shape(geom)); ids.append(int(val))

gdf_crowns = gpd.GeoDataFrame({"tree_id": ids}, geometry=geoms, crs=chm_crs)
gdf_crowns["crown_area_m2"] = gdf_crowns.geometry.area

# per-crown max height
lab = features.rasterize(list(zip(gdf_crowns.geometry, gdf_crowns["tree_id"])),
                         out_shape=chm_s.shape, transform=chm_tf, fill=0, dtype="int32")
max_h = np.zeros(labels.max()+1, dtype="float32")
for tid in np.unique(lab):
    if tid == 0: continue
    max_h[tid] = chm_s[lab==tid].max()
gdf_crowns["height_m"] = gdf_crowns["tree_id"].map(lambda t: float(max_h[int(t)]))

# export
gdf_pts.to_file(POINTS_GPKG, driver="GPKG", layer="Tree_Points_fromThreshold")
gdf_crowns.to_file(CROWNS_GPKG, driver="GPKG", layer="Tree_Crowns_fromThreshold")

# CSV with X/Y and height (join centroid to crowns by nearest tree_id label)
df_pts = gdf_pts.copy()
df_pts["X"] = df_pts.geometry.x
df_pts["Y"] = df_pts.geometry.y
df_pts = df_pts.merge(gdf_crowns[["tree_id","height_m","crown_area_m2"]], on="tree_id", how="left")
df_pts[["tree_id","X","Y","height_m","crown_area_m2"]].to_csv(POINTS_CSV, index=False)

print("Saved:\n ", POINTS_GPKG, "\n ", CROWNS_GPKG, "\n ", POINTS_CSV)


Saved:
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Points.gpkg 
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Crowns.gpkg 
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Points.csv


In [ ]:
# Non-maximum suppression across all tiles to merge duplicates
def iou(a,b):
    xA = max(a[0], b[0]); yA = max(a[1], b[1])
    xB = min(a[2], b[2]); yB = min(a[3], b[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    if inter <= 0: return 0.0
    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA + areaB - inter)

if len(df_boxes):
    boxes = df_boxes[["xmin","ymin","xmax","ymax"]].to_numpy().astype(float)
    scores= df_boxes["score"].to_numpy().astype(float)
    idxs  = scores.argsort()[::-1]
    keep  = []
    while len(idxs):
        i = idxs[0]
        keep.append(i)
        if len(idxs)==1: break
        rest = idxs[1:]
        ious = np.array([iou(boxes[i], boxes[j]) for j in rest])
        idxs = rest[ious < 0.3]  # IoU threshold
    df_boxes = df_boxes.iloc[keep].reset_index(drop=True)
    print("After NMS:", len(df_boxes))


After NMS: 2


In [ ]:
# --- Make a true 3-band RGB GeoTIFF from a 4-band source (RGBA or RGB+NIR) ---

from pathlib import Path
import os
import numpy as np
import rasterio as rio

# 1) Set your source path (the 4-band file). If you already have RGB_PATH defined, reuse it:
SRC_PATH = "/content/drive/MyDrive/GeoAI/Test1_DSM_DTM/Test1_Trees.tif"  # <-- change if different
OUT_DIR  = "/content/drive/MyDrive/GeoAI/out_colab"                      # output folder
os.makedirs(OUT_DIR, exist_ok=True)

RGB3_PATH = str(Path(OUT_DIR) / "RGB_3band.tif")

# 2) Read bands 1–3 as RGB and write a 3-band file
with rio.open(SRC_PATH) as src:
    if src.count < 3:
        raise ValueError(f"Source has {src.count} band(s). Need at least 3 (RGB).")
    R = src.read(1)
    G = src.read(2)
    B = src.read(3)

    # If your image is float 0–1, scale to 0–255 uint8 (DeepForest expects 0–255 float32 or uint8)
    def to_uint8(a):
        a = a.astype("float32")
        if a.max() <= 1.0:
            a = a * 255.0
        a[a < 0] = 0
        a[a > 255] = 255
        return a.astype("uint8")

    R8, G8, B8 = to_uint8(R), to_uint8(G), to_uint8(B)

    profile = src.profile.copy()
    profile.update(count=3, dtype="uint8", compress="LZW")

    with rio.open(RGB3_PATH, "w", **profile) as dst:
        dst.write(R8, 1)
        dst.write(G8, 2)
        dst.write(B8, 3)

print("Wrote 3-band RGB:", RGB3_PATH)
with rio.open(RGB3_PATH) as test:
    print("Bands in RGB3_PATH:", test.count)  # must be 3


Wrote 3-band RGB: /content/drive/MyDrive/GeoAI/out_colab/RGB_3band.tif
Bands in RGB3_PATH: 3


In [ ]:
import numpy as np, pandas as pd, rasterio as rio
from deepforest import main

# tuning
DF_SCORE      = 0.12     # lower -> more detections
PATCH_SIZE    = 512
PATCH_OVERLAP = 0.25     # 25% overlap
STRIDE        = int(PATCH_SIZE * (1 - PATCH_OVERLAP))

# read RGB3 into numpy
with rio.open(RGB3_PATH) as src:
    R = src.read(1).astype("float32"); R[R<0]=0
    G = src.read(2).astype("float32"); G[G<0]=0
    B = src.read(3).astype("float32"); B[B<0]=0
    transform = src.transform; crs = src.crs
    H, W = src.height, src.width

# scale up if 0–1
if R.max() <= 1 and G.max() <= 1 and B.max() <= 1:
    R *= 255; G *= 255; B *= 255
rgb_full = np.dstack([R,G,B]).astype("float32")

# model
model = main.deepforest()
model.use_release()

# tile + predict_image
all_dfs = []
for top in range(0, H, STRIDE):
    for left in range(0, W, STRIDE):
        bottom = min(top + PATCH_SIZE, H)
        right  = min(left + PATCH_SIZE, W)
        tile = rgb_full[top:bottom, left:right, :]
        if tile.shape[0] < 32 or tile.shape[1] < 32:
            continue
        df = model.predict_image(tile, return_plot=False)
        if df is None or len(df)==0:
            continue
        df = df.copy()
        df["xmin"] += left; df["xmax"] += left
        df["ymin"] += top;  df["ymax"] += top
        all_dfs.append(df)

import pandas as pd
df_boxes = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame(
    columns=["xmin","ymin","xmax","ymax","label","score"]
)
df_boxes = df_boxes[df_boxes["score"] >= DF_SCORE].reset_index(drop=True)
print(f"DeepForest detections kept @score≥{DF_SCORE}: {len(df_boxes)}")


Reading config file: /usr/local/lib/python3.12/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Reading config file: /usr/local/lib/python3.12/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


DeepForest detections kept @score≥0.12: 1002


In [ ]:
import numpy as np
def iou(a,b):
    xA = max(a[0], b[0]); yA = max(a[1], b[1])
    xB = min(a[2], b[2]); yB = min(a[3], b[3])
    inter = max(0, xB-xA) * max(0, yB-yA)
    if inter <= 0: return 0.0
    areaA = (a[2]-a[0])*(a[3]-a[1]); areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA + areaB - inter)

if len(df_boxes):
    boxes = df_boxes[["xmin","ymin","xmax","ymax"]].to_numpy().astype(float)
    scores= df_boxes["score"].to_numpy().astype(float)
    idxs  = scores.argsort()[::-1]
    keep  = []
    while len(idxs):
        i = idxs[0]; keep.append(i)
        if len(idxs)==1: break
        rest = idxs[1:]
        ious = np.array([iou(boxes[i], boxes[j]) for j in rest])
        idxs = rest[ious < 0.3]
    df_boxes = df_boxes.iloc[keep].reset_index(drop=True)
    print("After NMS:", len(df_boxes))


After NMS: 758


In [ ]:
import geopandas as gpd, rasterio as rio
from rasterio import features
from rasterio.transform import xy
from shapely.geometry import shape
from scipy.ndimage import gaussian_filter, maximum_filter
from skimage.segmentation import watershed
import numpy as np

# CHM params
MIN_HEIGHT_M    = 1.6     # relax to boost counts
GAUSS_SIGMA_PX  = 0.5     # lighter smoothing -> fewer merges
CROWN_DIAM_M_SET= [1.4, 1.7, 2.0]
MIN_CROWN_M2    = 0.25

# read CHM
with rio.open(CHM_PATH) as chm_src:
    chm = chm_src.read(1).astype("float32"); chm[chm<0]=0
    if GAUSS_SIGMA_PX>0: chm = gaussian_filter(chm, sigma=GAUSS_SIGMA_PX)
    chm_tf = chm_src.transform; chm_crs = chm_src.crs
    px_w, px_h = abs(chm_tf.a), abs(chm_tf.e)

mask = chm >= MIN_HEIGHT_M

# DF centers -> seeds
DF_cx = ((df_boxes["xmin"] + df_boxes["xmax"]) / 2.0).round().astype(int).to_numpy()
DF_cy = ((df_boxes["ymin"] + df_boxes["ymax"]) / 2.0).round().astype(int).to_numpy()

# peaks at multiple crown diameters
def peaks_for_diam_m(chm_arr, mask_arr, diam_m, px_m):
    win = max(3, int(round(diam_m / px_m)))
    if win % 2 == 0: win += 1
    filt = maximum_filter(chm_arr, size=win, mode="nearest")
    peaks = (chm_arr == filt) & mask_arr
    ry, rx = np.where(peaks)
    return np.vstack([ry, rx]).T

merged = np.zeros_like(chm, dtype=bool)
def add_points(arr_rc):
    cnt=0
    for r,c in arr_rc:
        if 0<=r<merged.shape[0] and 0<=c<merged.shape[1] and not merged[r,c]:
            merged[r,c]=True; cnt+=1
    return cnt

total = add_points(np.vstack([DF_cy, DF_cx]).T)
for d in CROWN_DIAM_M_SET:
    total += add_points(peaks_for_diam_m(chm, mask, d, px_w))
ry, rx = np.where(merged)
print("Total seeds (DF + peaks):", len(rx))

# watershed
markers = np.zeros_like(chm, dtype=np.int32)
for i,(r,c) in enumerate(zip(ry,rx), start=1):
    markers[r,c] = i
labels = watershed(-chm, markers=markers, mask=mask)

# drop tiny crowns
px_area = px_w * px_h
min_px = max(1, int(round(MIN_CROWN_M2 / px_area)))
counts = np.bincount(labels.ravel())
labels[np.isin(labels, np.where(counts < min_px)[0])] = 0

# vectorize + attributes
geoms, ids = [], []
for geom, val in features.shapes(labels.astype("int32"), mask=(labels>0), transform=chm_tf):
    if val==0: continue
    geoms.append(shape(geom)); ids.append(int(val))
gdf_crowns = gpd.GeoDataFrame({"tree_id": ids}, geometry=geoms, crs=chm_crs)
gdf_crowns["crown_area_m2"] = gdf_crowns.geometry.area

lab = features.rasterize(list(zip(gdf_crowns.geometry, gdf_crowns["tree_id"])),
                         out_shape=chm.shape, transform=chm_tf, fill=0, dtype="int32")
max_h = np.zeros(labels.max()+1, dtype="float32")
for tid in np.unique(lab):
    if tid==0: continue
    max_h[tid] = chm[lab==tid].max()
gdf_crowns["height_m"] = gdf_crowns["tree_id"].map(lambda t: float(max_h[int(t)]))

# centroids as points
gdf_points = gdf_crowns[["tree_id","height_m","crown_area_m2"]].copy()
gdf_points["geometry"] = gdf_crowns.geometry.centroid
gdf_points = gpd.GeoDataFrame(gdf_points, geometry="geometry", crs=chm_crs)

print("Counts — crowns:", len(gdf_crowns), "points:", len(gdf_points))


Total seeds (DF + peaks): 979
Counts — crowns: 258 points: 258


In [ ]:
import pandas as pd, os
os.makedirs(OUT_DIR, exist_ok=True)

gdf_crowns.to_file(OUT_CROWNS_GPKG, driver="GPKG", layer="Tree_Crowns_boost")
gdf_points.to_file(OUT_POINTS_GPKG, driver="GPKG", layer="Tree_Points_boost")

df_xy = gdf_points.copy()
df_xy["X"] = df_xy.geometry.x; df_xy["Y"] = df_xy.geometry.y
df_xy[["tree_id","X","Y","height_m","crown_area_m2"]].to_csv(OUT_POINTS_CSV, index=False)

print("Saved:\n ", OUT_CROWNS_GPKG, "\n ", OUT_POINTS_GPKG, "\n ", OUT_POINTS_CSV)


Saved:
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Crowns_boost.gpkg 
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Points_boost.gpkg 
  /content/drive/MyDrive/GeoAI/out_from_thresholds/Tree_Points_boost.csv
